In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
# install Kaggle API
! pip install -q kaggle

In [ ]:
# create a directory as kaggle
! mkdir -p ~/.kaggle

In [ ]:
#import KAGGLE API key to colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# copy API key to kaggle directory
! cp kaggle.json ~/.kaggle

In [ ]:
# disable the API key
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# import the dataset
! kaggle datasets download -d tongpython/cat-and-dog

100% 217M/218M [00:06<00:00, 35.1MB/s]
100% 218M/218M [00:06<00:00, 32.7MB/s]


In [ ]:
#unzip
! unzip -q /content/cat-and-dog.zip

In [ ]:
training_dir = '/content/training_set/training_set'
test_dir = '/content/test_set/test_set'

Building the model

In [ ]:
# reshape the images
img_shape = (128,128,3)

In [ ]:
#load pre trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
# freezing the model
base_model.trainable = False

## Defining the custom head for network

In [ ]:
base_model.output

<tf.Tensor 'out_relu/Relu6_4:0' shape=(None, 4, 4, 1280) dtype=float32>

In [ ]:
#dont flat but use Global Average max pooling
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_4/Mean:0' shape=(None, 1280) dtype=float32>

In [ ]:
#outputlayer
prediccation_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

Define the transfer learning model

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input,outputs=prediccation_layer)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
#compile mode
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [ ]:
model.compile(optimizer=opt, loss= 'binary_crossentropy' , metrics= ['accuracy'])

In [ ]:
#create data generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen_train = ImageDataGenerator(rescale= 1/255.0)
data_gen_test = ImageDataGenerator(rescale= 1/255.0)

In [ ]:
train_generator = data_gen_train.flow_from_directory(directory=training_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 8005 images belonging to 2 classes.


In [ ]:
test_generator = data_gen_test.flow_from_directory(directory=test_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2023 images belonging to 2 classes.


In [ ]:
#train the model
model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
63/63 [==============================] - 139s 2s/step - loss: 0.5050 - accuracy: 0.7650 - val_loss: 0.3860 - val_accuracy: 0.8680
Epoch 2/5
63/63 [==============================] - 136s 2s/step - loss: 0.3068 - accuracy: 0.9013 - val_loss: 0.2538 - val_accuracy: 0.9194
Epoch 3/5
63/63 [==============================] - 143s 2s/step - loss: 0.2166 - accuracy: 0.9300 - val_loss: 0.1909 - val_accuracy: 0.9412
Epoch 4/5
63/63 [==============================] - 138s 2s/step - loss: 0.1719 - accuracy: 0.9428 - val_loss: 0.1578 - val_accuracy: 0.9530
Epoch 5/5
63/63 [==============================] - 139s 2s/step - loss: 0.1474 - accuracy: 0.9485 - val_loss: 0.1377 - val_accuracy: 0.9585


In [ ]:
#fine tuning the model
base_model.trainable = True

In [ ]:
len(base_model.layers)

155

In [ ]:
fine_tune_at = 100


In [ ]:
#freeze the layers before 100

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False


In [ ]:
#compile
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)

Epoch 1/5
63/63 [==============================] - 257s 4s/step - loss: 0.1126 - accuracy: 0.9548 - val_loss: 0.0938 - val_accuracy: 0.9649
Epoch 2/5
63/63 [==============================] - 257s 4s/step - loss: 0.0216 - accuracy: 0.9938 - val_loss: 0.1458 - val_accuracy: 0.9624
Epoch 3/5
63/63 [==============================] - 249s 4s/step - loss: 0.0063 - accuracy: 0.9988 - val_loss: 0.1432 - val_accuracy: 0.9644
Epoch 4/5
63/63 [==============================] - 258s 4s/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 0.4449 - val_accuracy: 0.9362
Epoch 5/5
63/63 [==============================] - 253s 4s/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.2738 - val_accuracy: 0.9545


Overall accuracy after fine tuning is 99% and val accuracy is 93 %